# Linear Algebra

## Elimination

Elimination of linear algebra is the basic process of eliminating all the variables from a equation that you have 1 variable left. For example, if you have an equation with X, Y, Z variables, you can use the other equations to knockout X, Y variables such that you have Z left. 

In most text, they will teach to use a certain position as a pivot, but it really doesn't matter. Just do whatever it takes to eliminate the variables, you can subtract from any position as long as it serves the purpose in eliminating the variables.

Moreover, you can observe that once you have eliminated one equation to only have 1 variable, you can then use that equation to eliminate the other equations. For example, if you have an equation with 0X + 0Y + 2Z, this becomes a powerful tool to eliminate Z from the other 2 equations because X and Y is zero and has no effect. In the end, you keep doing eliminations until you have only 1 variable per equation.